<a href="https://colab.research.google.com/github/bhargav23/EVA4/blob/master/S15/EVA_S15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import sys
sys.path.append('/content/drive/My Drive/EVA4/S7')

In [0]:
import random
import time
from zipfile import ZipFile
import os
import numpy as np
import cv2

In [0]:
with ZipFile('/content/drive/My Drive/EVA4/data/fgbganddepth2.zip', 'r') as zipObj:
  zipObj.extractall()

In [0]:
path = "/content/drive/My Drive/EVA4/data/bgimages/"
bg = []
for p in os.listdir(path):
  for i in range(1000):
    bg.append(path+p)
bg.sort()


In [0]:
maskpath = "/content/output/masks/"
mask = []
for p in os.listdir(maskpath):
  mask.append(maskpath+p)
mask.sort()

In [0]:
fgbgpath = "/content/output/images/"
fgbg= []
for p in os.listdir(fgbgpath):
  fgbg.append(fgbgpath+p)
fgbg.sort()

In [0]:
depthpath = "/content/output/depth/"
depth= []
for p in os.listdir(depthpath):
  depth.append(depthpath+p)
depth.sort()

In [0]:
data = list(zip(bg[:10000], fgbg[:10000], mask[:10000], depth[:10000]))
random.shuffle(data)

In [0]:
import albumentations as A
from evaAlbumentations import AlbumentationTransforms

means = (0.49139968, 0.48215841, 0.44653091)
stdevs = (0.24703223, 0.24348513, 0.26158784)
train_bgtr = AlbumentationTransforms(
    [ A.Resize(128, 128),
      A.Normalize(mean=means, std=stdevs)])
train_fgbgtr = AlbumentationTransforms(
    [ A.Resize(128, 128),
      A.Normalize(mean=means, std=stdevs)])
train_masktr = AlbumentationTransforms(
    [ A.Resize(128, 128),
      A.Normalize(mean=means, std=stdevs)])
train_depthtr = AlbumentationTransforms(
    [ A.Resize(128, 128),
      A.Normalize(mean=means, std=stdevs)])


test_bgtr = AlbumentationTransforms(
    [ A.Resize(128, 128),
      A.Normalize(mean=means, std=stdevs)])
test_fgbgtr = AlbumentationTransforms(
    [ A.Resize(128, 128),
      A.Normalize(mean=means, std=stdevs)])
test_masktr = AlbumentationTransforms(
    [ A.Resize(128, 128),
      A.Normalize(mean=means, std=stdevs)])
test_depthtr = AlbumentationTransforms(
    [ A.Resize(128, 128),
      A.Normalize(mean=means, std=stdevs)])

In [0]:
import evadata 
from evadata import parrotDataset
trainlen = int(len(data)*0.7)
train = parrotDataset(data[:trainlen], train_bgtr,train_fgbgtr,train_masktr,train_depthtr )
test = parrotDataset(data[trainlen:], test_bgtr,test_fgbgtr,test_masktr,test_depthtr )

In [0]:
import evadataloader
dataloader = evadataloader.DataLoader()

trainloader = dataloader.load(train)
testloader = dataloader.load(test)

In [0]:
import evaresnet
from evaresnet import ResNet18


net = DepthMask()

In [0]:
from torchsummary import summary
import torch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = net.to(device)
summary(model, input_size=(3, 64, 64))

In [0]:
model

In [0]:
import sys
from torch_lr_finder import LRFinder
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-7, momentum=0.9, nesterov=True, weight_decay=1e-4)
lr_finder = LRFinder(model, optimizer, criterion, device="cuda")
lr_finder.range_test(trainloader, end_lr=100, num_iter=100, step_mode="exp")
# Plot learning rate vs loss
lr_finder.plot()

# Reset graph
lr_finder.reset()

In [0]:
loss = lr_finder.history['loss']
lr = lr_finder.history['lr']
max_lr = lr[loss.index(min(loss))]
max_lr

In [0]:
from torch.optim.lr_scheduler import StepLR
from torch.optim.lr_scheduler import ReduceLROnPlateau
from evascheduler import train,test
from torch.optim.lr_scheduler import OneCycleLR
L1lambda = 0
L2lambda = 0
EPOCHS = 50
model =  net.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, nesterov=True, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer,
                 max_lr=0.03,
                 total_steps=51,
                 epochs=51,
                 pct_start=0.2,
                 cycle_momentum=False,
                 base_momentum=0.85,
                 max_momentum=0.95,
                 div_factor=10.,
                 final_div_factor=1e4,
                 last_epoch=-1)

for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, trainloader, optimizer, epoch,criterion)
    scheduler.step()
    test(model, device, testloader)


In [0]:
from evascheduler import draw_graph
draw_graph()


In [0]:
from evascheduler import get_misclassified
misclassified = get_misclassified()

In [0]:
import utils
from utils import plot_images
plot_images(misclassified,classes,"misclassified.png")

In [0]:
model_save_name = 'models11.pt'
PATH = F"/content/drive/My Drive/EVA/S7/{model_save_name}" 
torch.save(model.state_dict(), PATH)

In [0]:
path = F"/content/drive/My Drive/EVA/S7/{model_save_name}"
model.load_state_dict(torch.load(path))